#### Teamname:
#### Score: 0.87571 Kagglename: M_Urban

#### Namen der Teammitglieder: Marius Urban, Ramon Jäckle, Maen Hanna

Standardcode aus Beispiel um Daten zu erhalten

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wget

In [ ]:
import os
import pandas as pd
import random
import zipfile

In [ ]:
!python -m wget https://filr.hs-offenburg.de/filr/public-link/file-download/0dcf8b85882ae199018870bceddf437c/5426/-2254907074900866378/test_data.zip

In [ ]:
!python -m wget https://zenodo.org/record/7869954/files/products_leaflets_256.zip?download=1

In [ ]:
path_to_zip_file = 'products_leaflets_256.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [ ]:
path_to_zip_file = 'test_data.zip'
directory_to_extract_to = '.'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

## Bildgröße und Datenverzeichnisse

1. Zuerst werden die Verzeichnisse für die Trainings- und Validierungsdaten festgelegt. Das Verzeichnis **train_directory** enthält die Bilder, die für das Training verwendet werden, und das Verzeichnis **val_directory** enthält die Bilder für die Validierung.

2. Die Bildgröße (**image_size**) wird auf (**256, 256**) festgelegt, was die Größe ist, auf die alle Bilder während des Trainings und der Validierung skaliert werden. Für Bilder muss eine konstante Größe gewählt werden, da dies von den Netzwerken erwartet werden, aber die Bilder aus den Prospekten nicht immer gleich groß sind. **Anmerkung: evtl. Warum größe gewählt**

3. Die Anzahl der Klassen (**num_classes**) wird automatisch festgelegt, indem die Anzahl der Unterordner (**Klassen**) im Trainingsverzeichnis gezählt wird. Jeder Unterordner im Trainingsverzeichnis repräsentiert eine Klasse von Produkten.

In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Verzeichnis mit den Trainingsdaten
train_directory = 'products_leaflets_256/train'
val_directory = 'products_leaflets_256/test'

# Bildgröße und Anzahl der Klassen
#
image_size = (256, 256)
num_classes = len(os.listdir(train_directory))

## Laden der Bilder
Es würde die Funktion **tf.keras.utils.image_dataset_from_directory** von TensorFlow/Keras, um Trainings- und Validierungsdatasets aus den entsprechenden Verzeichnissen zu erstellen, verwendet.
Diese Funktion bietet viele möglichkeiten um die daten zu laden und verarbeiten.
Für das Trainings- und Validierungsdaten wird die Funktion aufgerufen und mit den entsprechenden Parametern konfiguriert. Das Datenverzeichnis wird angegeben, sowie einige Einstellungen für die Datenverarbeitung:

  * **labels="inferred"**: Die Zielvariablen (Labels) werden automatisch aus den    Verzeichnisnamen abgeleitet. Jeder Unterordner im Trainingsverzeichnis repräsentiert eine Klasse.
  * **label_mode="int"**: Die Zielvariablen werden als Integer (ganzzahlige Werte) dargestellt, da sie nicht One-Hot-kodiert sind.
  * **color_mode="rgb"**: Die Bilder werden im RGB-Farbraum geladen, was üblicherweise bei den meisten Bilderkennungs-Aufgaben der Fall ist.
  * **batch_size=32**: Die Bilder werden in Batches mit einer Größe von 32 geladen, um den Speicher effizient zu nutzen.
  * **image_size=image_size**: Die Bilder werden auf die vorher festgelegte Bildgröße (256, 256) skaliert, um eine einheitliche Größe für das Modelltraining zu gewährleisten.
  * **shuffle=True**: Die Bilder werden vor dem Training zufällig durchmischt, um die Daten während des Trainings besser zu mischen und ein besseres Training zu ermöglichen. **Anmerkung: vlt noch beschreiben was am TRaining besser wird.**

In [ ]:
# Durchlaufe das Trainingsverzeichnis und lade die Bilder
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

# Durchlaufe das Validierungsverzeichnis und lade die Bilder
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,)

### Funktionen Impotieren

In [ ]:

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet169
import tensorflow.keras.optimizers.legacy as legacy_optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

## Netzauswahl

EfficientNetB0, EfficientNetB0, ResNet50, ResNet100, MobileNetV2, selbstgebautes Alexnet, DenseNet121, DenseNet169, DenseNet201.
**Anmerkung sollen Netze zwei mal aufgelistet werden?**
Es wurden verschiedene von Keras bereitgestellte Bilderkennungsnetze wie (EfficientNetV2B0, EfficientNetV2B0, ResNet50, ResNet101, MobileNetV2, selbstgebautes Alexnet, DenseNet121, DenseNet169, DenseNet201) getestet.

DenseNet169 schnitt am besten ab und wurde deshalb gewählt. Außerdem wurde diese ansatz verwendet, da bei DensNet169 dichte Verbindung von Schichten ermöglicht eine bessere Nutzung von Merkmalen aus früheren Schichten. Desweiteren kann DensNet169 komplexere Merkmale in tieferen Schichten lernen.

**Beschreibung der Lösung:** **Anmerkung: Nicht in Wir Form neutraler Stil**
  1. Wir laden das vortrainierte Modell DenseNet169 ohne die Ausgangsschicht mit **include_top=False** und **weights='imagenet'**. Damit erhalten wir das Modell ohne die Klassifikationsschicht. Wir können es dann an unser Problem anpassen.
  2. Durch Setzen von **layer.trainable = False** für jede Schicht frieren wir alle Schichten des Basismodells ein. Das bedeutet, dass die Basismodell-Gewichte nicht aktualisiert werden und wir uns darauf konzentrieren können, die neuen Ausgangsschichten zu trainieren.
  3. Wir fügen neue Ausgangsschichten hinzu, um das Modell für unsere spezifische Aufgabe anzupassen. In unserem Fall fügen wir eine finale Dense-Schicht mit der Anzahl der Klassen hinzu.
  4. Das Modell wird mit einer niedrigen Lernrate von 1e-4 kompiliert, um das Risiko von starken Änderungen der vortrainierten Gewichte zu verringern. Außerdem wird während der laufzeit durch die funktion **ReduceLROnPlateau** die Lernrate reduziert, um bessere ergebnisse zu erzielen.**Anmerkung: Begründung komisch in 3 saGST DU du hast die vortrainierten layer nicht trainierbar gemacht, hier sagts du die vortrainierten Gewichte sollen nicht durch eine zu große Lernrate verändert werden**


In [ ]:
# Modellarchitektur anpassen

#base_model = MobileNetV2(include_top=False, input_shape=(256, 256, 3))
#base_model = DenseNet121(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
base_model = DenseNet169(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
#base_model = DenseNet201(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))
#base_model = ResNet50(include_top=False, pooling='avg', weights = 'imagenet', input_shape=(256, 256, 3))

## Data Augmentation
Bei der Datenagumentation werden verschiedene Transformationen auf die vorhandenen Trainingsdaten angewendet. Dadurch werden neue, leicht veränderte Versionen der Bilder erzeugt, die jedoch dieselben Merkmale und Klassen enthalten. Diese künstlich erzeugten Daten können dem Modell dabei helfen, eine bessere Generalisierungsfähigkeit zu entwickeln, indem sie es in die Lage versetzen, Variationen von Bildern besser zu erkennen und zu verarbeiten.
Es wurden folgende Augmentations getestet:

1. Zufällige Rotation des Bildes
2. Zufälliges Heran-Zoomen
3. Zufälliges horizontales spiegeln
4. Zufällige Kontraständerungen

Ausnahmslos alle dieser Masnahmen verschlechterten das Ergebnis in DensNet169 und wurden daher verworfen.

In [ ]:
# Daten Augmentation
#data_augmentation = tf.keras.Sequential([
    #tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    #tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    #tf.keras.layers.RandomFlip(mode = 'horizontal'),
    #tf.keras.layers.RandomContrast(0.4)
#])

## Modelkonstruktion
Bei der Konstruktion des Gesamtmodells wurden folgende Dinge getestet:
1. Wie bereits weiter oben erläutert, handelt es sich bei der **data_augmentation** um eine Sequenz von Transformationen, die auf die Bilder des Trainingsdatensatzes angewendet werden, um neue, leicht veränderte Versionen der Bilder zu erzeugen. Diese Layer wurde verwurfen, da bei unsere Ansatz das nicht gebracht hat.
2. Mehrere **Dense_Layer** mit einer unterschiedlichen Anzahl von Neuronen. Dies führte zu einem schlechteren Ergebnis. Daher wurden diese Layer verworfen. **Anmerkung:warum diese Layer probiert?**
3. Die finale **Dense_layer**, die die Ausgabe des Modells darstellt. Die Anzahl der Neuronen entspricht der Anzahl der Klassen im Datensatz, und die Softmax-Aktivierungsfunktion wird verwendet, um eine Wahrscheinlichkeitsverteilung über die verschiedenen Klassen zu berechnen.

In [ ]:
# Data Augmentation erweitern
model = Sequential([
    #data_augmentation,
    base_model,
    #Dense(256, activation="relu"),
    #Dense(128, activation='relu'),
    #Dense(64, activation="relu"),
    Dense(num_classes, activation='softmax')
])
base_model.layers[0].trainable = False

## Anpassbare Lernrate
**ReduceLROnPlateau**: ist ein Callback in Keras, der automatisch die Lernrate des Optimierers anpasst, wenn ein bestimmtes Plateau erreicht wird, um die Validierungsverlustfunktion (val_loss) zu verbessern. Das heißt, wenn sich die Modellleistung auf dem Validierungsdatensatz nicht weiter verbessert, wird die Lernrate reduziert(factor = 0.2), um zu versuchen, in kleineren Schritten zu lernen.

Diese Methode führte zu masiven Verbesserung des Ergebnis.

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=1e-6)

## Optimimizer
Es wurden 2 ansätze gestestet:
  + **Adam**: ist ein häufig verwendeter Optimierungsalgorithmus für das Training von neuronalen Netzwerken. Diese erzielte bei der DenseNet169 ein sehr gutes ergebniss.
  + **legacy_optimizer SGD**: Diese Optimierungsalgorithmus wuerde auf das DensNet169 verwendet, aber führte zu schlechtere Ergebnisse.

In [ ]:
optimizer = Adam(learning_rate=1e-4)

In [ ]:
#sgd = legacy_optimizers.SGD(lr = 0.01, momentum = 0.9, nesterov = True)

## Vorzeitiges Stoppen

Early Stopping hilft dabei, zu verhindern, dass das Modell Rauschen in den Trainingsdaten auswendig lernt, und fördert das Lernen allgemeiner Muster. Es kann auch Rechenressourcen und Zeit sparen, da das Training gestoppt wird, sobald das Modell ein geeignetes Leistungsniveau erreicht hat, anstatt für die maximale Anzahl von Trainingsepochen ausgeführt zu werden.Es ist nützlich, um das Übertraining (Overfitting) zu verhindern und die besten Modellgewichte zu speichern.

Auswahl der Parameter:


1.   **monitor='val_loss'**: Die Metrik, die überwacht wird, um zu entscheiden, wann das Training gestoppt werden soll.
2.   **mode='min'**: Die Art der Überwachung. In diesem Fall wird nach einer Minimierung des Validierungsverlustes gesucht.
3. **patience=10**: Die Anzahl der Epochen, die gewartet werden sollen, wenn keine Verbesserung des Validierungsverlustes festgestellt wird.
4. **restore_best_weights=True**: Wenn True, wird das Modell nach dem Training auf die Gewichte zurückgesetzt, die den niedrigsten Validierungsverlust erzielt haben.
5. **verbose=1**: Das Level der Ausgaben während des Trainings. Wenn 1, werden Informationen über das Training und die Beendigung des Trainings angezeigt.



In [ ]:
EarlyStopping_Cb = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 10, restore_best_weights = True, verbose = 1)

## Modelkompilierung

Zur Modelkompilierung kann erwähnt werden, dass die Wahl der Metriken das Ergebnis verbessern kann.
So wurde hier das Ergebnis verbessert durch indem neben der Accuracy auch der Mean-Squared Error evaluiert wurde.

## Modeltraining

Das Model wurde in 25 Epochen trainiert, da hier bereits das Model innerhalb dieser garantiert durch den vorzeitigen Stop beendet wurde.
Es wurde eine batch_size von 32 gewählt, da damit bisher die besten Ergebnisse erzielt wurden.

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy', 'mse'])

# Trainingsparameter anpassen
batch_size = 32
epochs = 25
# Modelltraining mit erweitertem Data Augmentation
model.fit(train_ds,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=val_ds,
          callbacks=[reduce_lr, EarlyStopping_Cb])



## Prediction und speichern der Ergebnisse
Das Testverzeichnis wird durchlaufen, und für jedes Testbild wird eine Vorhersage mit dem Modell durchgeführt. Die Vorhersagen werden in einer Liste **label_predictions** gespeichert, wobei jedem Testbild der entsprechende Klassenname zugeordnet wird.

Anschließend wird ein leeres Pandas DataFrame df erstellt, das zwei Spalten enthält: "**image_id**" für die Dateinamen der Testbilder und "**label**" für die Vorhersagen.

Die Liste der Dateinamen der Testbilder (**image_names**) und die Liste der Vorhersagen (**label_predictions**) werden dem DataFrame zugewiesen.

Schließlich wird das DataFrame in eine CSV-Datei mit dem Namen "**submission.csv**" gespeichert.


In [ ]:
# Liste für Bildnamen und Vorhersagen
image_names = os.listdir('./test_data')
class_names = sorted(os.listdir(train_directory))
label_predictions = []
img_array = []
test_directory = 'test_data'


# Durchlaufe das Testverzeichnis und lade die Bilder
for image_name in image_names:
    url = "./test_data/" + image_name
    img = tf.keras.utils.load_img(url, target_size=image_size)
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    label_predictions.append(class_names[np.argmax(score)])


# Erstelle den DataFrame und speichere die Vorhersagen in einer CSV-Datei
df = pd.DataFrame(columns=['image_id', 'label'])
df['image_id'] = image_names
df['label'] = label_predictions
df.to_csv('submission.csv', index=False)

## Ergebnisanalyse